<a href="https://colab.research.google.com/github/aghapygad336/MENA/blob/master/MENA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
#import necessary modules
import csv
with open('2015.csv','rt')as f:
  data = csv.reader(f)
print(type(f))        

<class '_io.TextIOWrapper'>


In [31]:
def reencode(file):
    for line in file:
        yield line.decode('windows-1250').encode('utf-8')

csv_reader = csv.reader(reencode(open("TDS2017.csv")), delimiter=";",quotechar='"')
import csv  
reader = csv.reader(open(csv_reader, mode = 'r'))  
for row in reader:  
    print (row)  

TypeError: ignored